### Задача - посчитать среднее время жизни пользователей, которые выставили более 100 оценок

Используем файл с оценками фильмов ml-latest-small/ratings.csv.  
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [3]:
# Исходный датафрейм
df3 = pd.read_csv('ml-latest-small/ratings.csv')
df3.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#### Решение 1.

In [4]:
df4 = df3.groupby('userId').agg({'timestamp': ['min', 'max', 'count']})
df4.head()

timestamp                  
               min         max count
userId                              
1        964980499   965719662   232
2       1445714835  1445715340    29
3       1306463323  1306464293    39
4        945078428  1007574542   216
5        847434747   847435337    44

In [5]:
# Оставляем (фильтруем) в датафрейме только пользователей, которые поставили более 100 оценок
df5 = df4.loc[df4['timestamp']['count'] > 100]['timestamp'].sort_values('count')
df5.head()

,min,max,count
userId,,,
601,1441639169,1521467863,101
412,939113589,939563056,102
32,856736119,856737388,102
593,1181007064,1181008767,103
40,832058854,832060442,103


In [6]:
df5['lifetime'] = df5['max'] - df5['min']
df5.sort_values('lifetime').head()

,min,max,count,lifetime
userId,,,,
488,1112381806,1112382828,111,1022
52,1468051091,1468052169,130,1078
230,1196304098,1196305204,139,1106
465,959895446,959896638,115,1192
117,844162743,844163992,165,1249


In [7]:
df5['lifetime'] = df5['lifetime'].div(31536000)
df5.head()

,min,max,count,lifetime
userId,,,,
601,1441639169,1521467863,101,2.531351
412,939113589,939563056,102,0.014253
32,856736119,856737388,102,0.000040
593,1181007064,1181008767,103,0.000054
40,832058854,832060442,103,0.000050


In [8]:
# Находим среднее время жизни пользователей, которые поставили более 100 оценок, в годах
df5.lifetime.mean()

1.418422366016422

####  Решение 2.

In [9]:
# Находим максимальную дату для каждого пользователя
df_max = df3.groupby('userId').max()[['timestamp']]
df_max.rename(columns = {'timestamp':'timestamp_max'}, inplace = True)
df_max.head()

,timestamp_max
userId,
1,965719662
2,1445715340
3,1306464293
4,1007574542
5,847435337


In [10]:
# Находим минимальную дату для каждого пользователя
df_min = df3.groupby('userId').min()[['timestamp']]
df_min.rename(columns = {'timestamp':'timestamp_min'}, inplace = True)
df_min.head()

,timestamp_min
userId,
1,964980499
2,1445714835
3,1306463323
4,945078428
5,847434747


In [11]:
# Объединяем максимальную и минимальную даты для каждого пользователя
df_new = df_min.join(df_max)
df_new.head()

,timestamp_min,timestamp_max
userId,,
1,964980499,965719662
2,1445714835,1445715340
3,1306463323,1306464293
4,945078428,1007574542
5,847434747,847435337


In [12]:
# Находим время жизни для каждого пользователя
df_new['lifetime'] = df_new['timestamp_max'] - df_new['timestamp_min']
df_new.head()

,timestamp_min,timestamp_max,lifetime
userId,,,
1,964980499,965719662,739163
2,1445714835,1445715340,505
3,1306463323,1306464293,970
4,945078428,1007574542,62496114
5,847434747,847435337,590


In [13]:
# Считаем количество оценок для каждого пользователя
df4 = df3.groupby('userId').count()
df4.head()

,movieId,rating,timestamp
userId,,,
1,232,232,232
2,29,29,29
3,39,39,39
4,216,216,216
5,44,44,44


In [14]:
# Оставляем (фильтруем) в датафрейме только пользователей, которые поставили более 100 оценок
df5 = df4.loc[df4['rating'] > 100].sort_values('rating')
df5.head()

,movieId,rating,timestamp
userId,,,
601,101,101,101
412,102,102,102
32,102,102,102
593,103,103,103
40,103,103,103


In [15]:
# Соединяем данные пользователей с их временем жизни
joined = df5.merge(df_new, how = 'left', on = 'userId')
joined.head()

,movieId,rating,timestamp,timestamp_min,timestamp_max,lifetime
userId,,,,,,
601,101,101,101,1441639169,1521467863,79828694
412,102,102,102,939113589,939563056,449467
32,102,102,102,856736119,856737388,1269
593,103,103,103,1181007064,1181008767,1703
40,103,103,103,832058854,832060442,1588


In [16]:
# Переводим секунды в годы
joined['lifetime'] = joined['lifetime'].div(31536000)

In [17]:
# Находим среднее время жизни пользователей, которые поставили более 100 оценок, в годах
joined.lifetime.mean()

1.418422366016422